In [ ]:
%matplotlib inline

# Get the PODPAC logger
import logging
logger = logging.getLogger("podpac")

# Introduction

This notebook demonstrates how to run PODPAC Pipeline on AWS Lambda. Specifically, we will:
* Create a node
* Execute the node on AWS Lambda

**This example requires a pre-configured AWS Lambda node.** We will not cover how the AWS Lambda node was created. For more details see the [AWS Lambda Tutorial](../developer/aws-lambda-tutorial.ipynb). 

<small>*For instructions on using Jupyter notebooks, see the [README.md](../../README.md) file.*</small>

# Setup

Make sure you have PODPAC installed with the **datatype** and **aws** optional dependencies:

```
$ pip install podpac[datatype]
$ pip install podpac[aws]
```

# How to run analysis on AWS Lambda

<i style='font-size:18pt'>PODPAC uses the Python package `boto3` to interact with an S3 bucket which triggers the Lambda function execution. It also uses `boto3` to monitor an S3 bucket for the computed output. </i>


<img src='../../images/PODPAC-AWS-Lambda.png' style='width:80%; margin-left:auto;margin-right:auto' />

# Setup PODPAC for AWS (1/2)

* Configure AWS credentials
* Can be specified at runtime, or in the podpac settings file

In [ ]:
import podpac
from podpac import settings

# need to allow unsafe evaluation to use `podpac.algorithm.Arithmetic` Node (below) 
settings.set_unsafe_eval(True)

# # Credentials
# settings["AWS_ACCESS_KEY_ID"] = "access key id"
# settings["AWS_SECRET_ACCESS_KEY"] = "secrect access key"
# settings["AWS_REGION_NAME"] = "region name"

# # General Settings
# settings["AWS_TAGS"] = {} # tags to assign to AWS resources created through PODPAC

# # S3
# settings["S3_BUCKET_NAME"] = "S3 bucket for Lambda functions or PODPAC cache"

# # Lambda
# settings["FUNCTION_NAME"] = "name of lambda function to eval"
# settings["FUNCTION_ROLE_NAME"] = "role name for lambda function"
# settings["FUNCTION_DEPENDENCIES_KEY"] = "path on S3 bucket where function depedencies live"
# settings["FUNCTION_S3_INPUT"] = "path on S3 bucket for input pipelines. Objects put in this directory will trigger lambda function",
# settings["FUNCTION_S3_OUTPUT"] = "path on S3 bucket for pipeline outputs. Objects put in this directory will be returned to lambda function",

# # Paths - overwrite paths for Lambda caching
# this will be fixed in future releases of PODPAC
settings["ROOT_PATH"] = "/tmp/"
settings["LOG_FILE_PATH"] = "/tmp/"

## Provide Earth Data Login Credentials
If you do not have an earth data login, or have not activated OpenDAP access, follow the [instructions here](https://creare-com.github.io/podpac-docs/user/earthdata.html).

In [ ]:
import getpass
username = password = None
username = input("Username:");   password = getpass.getpass('Password:')

# EarthData Credentials need to get saved in `settings` in order to get passed
# into the Lambda function on AWS. This will be automated in the future.
settings["username@urs.earthdata.nasa.gov"] = username
settings["password@urs.earthdata.nasa.gov"] = password

# Setup (2/2)

* Create the PODPAC Pipeline
* We'll use the same pipeline from the [100-analyzing-SMAP-data.ipynb](100-analyzing-SMAP-data.ipynb) notebook
* This example computes the difference between the current soil moisture for a region, and that of the previous year

In [ ]:
import podpac.datalib

# Create the Pipeline
# See 100-analyzing-SMAP-data.ipynb for more details
product = 'SPL4SMAU'
smap = podpac.datalib.SMAP(product=product, username=username, password=password)
smap_time1_offset = podpac.algorithm.ExpandCoordinates(source=smap, time=['-1,Y', '-1,Y'])
smap_offset = podpac.algorithm.Mean(source=smap_time1_offset, dims=['time'])

# This is the output Node of the Pipeline
diff = podpac.algorithm.Arithmetic(eqn='B-A', A=smap, B=smap_offset)

# Create PODPAC Coordinates 
* This specifies the region and date where the pipeline will be evaluated

In [ ]:
# Specify region of interest on a uniform grid
lat = podpac.crange(  60,  10, -2.0)  # (start, stop, step)
lon = podpac.crange(-130, -60,  2.0)  # (start, stop, step)

# Specify date and time
time = '2018-05-19T12:00:00'

# Create the PODPAC Coordinates
coords = podpac.Coordinates([lat, lon, time], dims=['lat', 'lon', 'time'])

# Evaluating node on AWS cloud

In [ ]:
# Set up wrapper to run your node on AWS Lambda
node = podpac.managers.aws.Lambda(source=diff, function_allow_unsafe_eval=True)

In [ ]:
# if this is the first time your are using this node, build the AWS resources
# if you have used this function before, skip this step
node.build()

In [ ]:
# validate that the AWS resources are built
node.validate()

# review aws resources
node.describe()

In [ ]:
# Run the node (on AWS Lambda) and get the results back
# (the first time this runs will be slower than subsequent calls)
output = node.eval(coords)
output.plot()
pass